# First get all of the data. 

In [8]:
# first get all X and y data from the all_points_folds
import numpy as np 
import pickle, random 
import cv2, os

X_all, y = [], []  # X needs to be changed for each fold, y doesn't need to be changed for each fold. 

for file in os.listdir("all_points_folds"): 
    with open(f"all_points_folds/{file}", 'rb') as f: 
        X_y = pickle.load(f)
        X_all.append(X_y[0])
        y.append(X_y[1])

        
def specify_data(dataset, landmarks): 
    ds = np.concatenate([X_i for X_i in dataset])
    """
    dataset should contain a 4D matrix of (5, _, 90, 126). 
    """ 
    cols = []
    for landmark in landmarks:
        cols += (np.array([0, 21, 42, 63, 84, 105]) + landmark).tolist()
    
    return ds[:, :, tuple(cols)]

X_six = specify_data(X_all, [0, 4, 8, 12, 16, 20]).reshape(5, 20, 90, 36)
X_one = specify_data(X_all, [0]).reshape(5, 20, 90, 6) # just take the first landmark 


import random 
def shuffle(X, y, seed = None):
    if seed == None:  
        seed = random.randrange(0, 100)
        print(f"using seed {seed}")
    np.random.seed(seed) 
    new_X = np.concatenate([X_i for X_i in X])
    new_y = np.concatenate([y_i for y_i in y])
    N = np.random.permutation(new_X.shape[0])
    new_X = new_X[N]
    new_y = new_y[N]
    new_X = new_X.reshape(5, 20, 90, new_X.shape[-1])
    new_y = new_y.reshape(5, 20)
    return new_X, new_y

In [9]:
SEED = 65
X_all, y = shuffle(X_all, y, seed = SEED)
X_six, _ = shuffle(X_six, y, seed = SEED)
X_one, _ = shuffle(X_one, y, seed = SEED)

In [10]:
def ensemble_val_acc(models, X_tests, y_test): 
    preds = np.zeros_like(y_test)
    for model, X_test in zip(models, X_tests): 
        preds += model.predict(X_test).flatten() 
    preds = preds / len(models)
    return (np.round_(preds) == y_test).mean()

In [20]:
import time 
def cross_validate(make_model_all, make_model_six, make_model_one, X_all, X_six, X_one, y, epochs = 75, callbacks = []): 
    for i in range(5): 
        model_all = make_model_all() 
        model_six = make_model_six()
        model_one = make_model_one() 
        
        # define global labels
        y_train = np.concatenate([y_j for j, y_j in enumerate(y) if i != j])
        y_test = y[i]
        
        # first run all 
        X_test_all = X_all[i]
        X_train_all = np.concatenate([X_j for j, X_j in enumerate(X_all) if i != j])
        
        try:
            os.remove("best_aso.h5") 
        except Exception as e: 
            pass 

        # train 
        history_all = model_all.fit(X_train_all, y_train, validation_data = (X_test_all, y_test), epochs = epochs, callbacks = callbacks)
        
        try: 
            model_all.load_weights("best_aso.h5")
        except Exception as e: 
            pass
        
        print("\nevaluation on all:")
        model_all.evaluate(X_test_all, y_test)
        
        time.sleep(1)
        # next train six 
        
        X_test_six = X_six[i]
        X_train_six = np.concatenate([X_j for j, X_j in enumerate(X_six) if i != j])
        
        try:
            os.remove("best_aso.h5") 
        except Exception as e: 
            pass 

        # train 
        history_six = model_six.fit(X_train_six, y_train, validation_data = (X_test_six, y_test), epochs = epochs, callbacks = callbacks)
        
        try: 
            model_six.load_weights("best_aso.h5")
        except Exception as e: 
            pass
        
        print("\nevaluation on six:")
        model_six.evaluate(X_test_six, y_test)
        time.sleep(1)
        
        # next train one
        X_test_one = X_one[i]
        X_train_one = np.concatenate([X_j for j, X_j in enumerate(X_one) if i != j])
        
        try: 
            os.remove("best_aso.h5")
        except Exception as e: 
            pass 
    
        # train this model 
        history_one = model_one.fit(X_train_one, y_train, validation_data = (X_test_one, y_test), epochs = epochs, callbacks = callbacks)
        
        try: 
            model_six.load_weights("best_aso.h5")
        except Exception as e: 
            pass
        
        print("\nevaluation on one:")
        model_one.evaluate(X_test_one, y_test)
        time.sleep(1)
        
        # YAY! WE HAVE TRAINED THE MODEL ON EVERYTHING FOR THIS FOLD
    
        # get the aggregate validation accuracy on everything
        models = [model_all, model_six, model_one]
        print(f"ensemble validation accuracy : {ensemble_val_acc(models, [X_test_all, X_test_six, X_test_one], y_test)}")
        time.sleep(2)

In [12]:
# create the functions to create models 
import tensorflow as tf 
def make_model_all(): 
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=False), 
        tf.keras.layers.Dropout(0.3), 
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])

    model.compile(loss = "binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model

def make_model_six(): 
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=False, input_shape = (None, 36)), 
        tf.keras.layers.Dropout(0.3), 
        tf.keras.layers.Dense(1, activation = 'sigmoid') 
    ]) 

    model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate=0.01), metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

def make_model_one(): 
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(32, return_sequences=False), 
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])
    
    model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate=0.01), metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [21]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("best.h5", save_best_only=True, monitor = "val_accuracy")
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", patience=10)
cross_validate(make_model_all, make_model_six, make_model_one, X_all, X_six, X_one, y, epochs = 75, callbacks=[checkpoint, early_stopping])

Epoch 1/75
3/3 [==============================] - 2s 283ms/step - loss: 0.6967 - accuracy: 0.4500 - precision_30: 0.4839 - recall_30: 0.7143 - val_loss: 0.7029 - val_accuracy: 0.4000 - val_precision_30: 0.4000 - val_recall_30: 1.0000
Epoch 2/75
3/3 [==============================] - 0s 31ms/step - loss: 0.6775 - accuracy: 0.5625 - precision_30: 0.5479 - recall_30: 0.9524 - val_loss: 0.6924 - val_accuracy: 0.4500 - val_precision_30: 0.4211 - val_recall_30: 1.0000
Epoch 3/75
3/3 [==============================] - 0s 31ms/step - loss: 0.6673 - accuracy: 0.6750 - precision_30: 0.6290 - recall_30: 0.9286 - val_loss: 0.6820 - val_accuracy: 0.6500 - val_precision_30: 0.5455 - val_recall_30: 0.7500
Epoch 4/75
3/3 [==============================] - 0s 30ms/step - loss: 0.6513 - accuracy: 0.6750 - precision_30: 0.6379 - recall_30: 0.8810 - val_loss: 0.6697 - val_accuracy: 0.7000 - val_precision_30: 0.6000 - val_recall_30: 0.7500
Epoch 5/75
3/3 [==============================] - 0s 32ms/step - lo